In [1]:
#This notebook shows how algorithms are called and utilized to rank instances.
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Nov  3 14:09:30 2019

@author: can
"""

#please type the location where github_column_generation folder exists.
folder_location = "C:\\Users\\PC\\Desktop\\column_generation_generalized_version"
#select the dataset you want to work
#xor,xor_two,votes,survival,spectf,sonar,spambase,ionosphere,cancer_wbc,monks1,cleveland_heart....
d_name="xor"

In [2]:
folder_location=folder_location+"\\github_column_generation"
data_location=folder_location+"\\data"
script_location=folder_location+"\\scripts"
import os
import datetime
import math
import pandas as pd
from scipy.spatial import distance_matrix
import numpy as np
from gurobipy import *
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
os.chdir(script_location)
from read_available_datasets import selected_data_set
from sklearn import tree
import matplotlib.pyplot as plt
from scipy import signal
import scipy
from scipy import stats
from scipy.stats import iqr
import itertools
import random
import cvxpy as cp
from cvxpy.atoms.pnorm import pnorm
#import tensorflow as tf

In [3]:
#selected_data_set function applies different pre processing operations to datasets mentioned in the "A Column Generation to 
#Solve Ranking Problems" paper. Therefore, to obtain similar results, this function must be called before running the algorithm.

df,df_test,test_class,test_data,train_class,train_data=selected_data_set(datasetname=d_name,location=data_location)
random.seed(3)
data=train_data.append(test_data)           
class_data=train_class.append(test_class) 

#After running this function method can be called as it is shown below. The summarized performance measures in the paper
#are obtained by using 10-fold cross validation with 5 repetitions. While splitting data into folds stratification technique is
#utilized. Test indices and Training indices for each fold are given for each data set to promote reproducibility. 
#After callling selected_data_set function, you can read train and test indices from corresponding text files and obtain
#necessary variables such as df, df_test,test_class,test_data,train_data, and train_class 
#by using data, and class_data variables. Then, the performance of models can be observed.

In [4]:
os.chdir(script_location)
from ranking_models import rankSVM_column_generation



        
method1=rankSVM_column_generation(train_data,train_class,test_data,test_class,df,df_test,
                          distance="euclidian",stopping_condition="obj",
                          stopping_percentage=0.005,
                          selected_col_index=0,scale=True,ignore_initial=False,manual_selection=False)

method1.data_preprocess()
method1.solve_full_model()
method1.predict_test_full_model()
        





C:\Users\PC\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\PC\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\PC\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\PC\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1)

Academic license - for non-commercial use only
model constraints are being counstructed
end
Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Crossover to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 42 rows, 55 columns and 588 nonzeros
Coefficient statistics:
  Matrix range     [2e-02, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 42 rows, 55 columns, 588 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.339722e+01   0.000000e+00      0s
      10    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds
Optimal objective  0.000000000e+00


C:\Users\PC\Desktop\column_generation_generalized_version\github_column_generation\scripts\ranking_models.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_data['counter'] = range(len(self.df))


In [5]:
tr_accu_final=method1.ftrain_accuracy
tr_auc_final=method1.ftrainroc
test_accu_final=method1.ftest_accu
test_auc_final=method1.ftestroc     


print("Test AUC:",test_auc_final)
print("Test Accuracy:",test_accu_final)
print("Train AUC:",tr_auc_final)
print("Train Accuracy:",tr_accu_final)

Test AUC: 0.8753462603878116
Test Accuracy: 0.7894736842105263
Train AUC: 1.0
Train Accuracy: 1.0


In [ ]:
"""
10-fold cross validation with 5 repetition can be applied by using this code chunk. Since code reads 
indices from a text file, results summarized in the paper can be obtained.
"""
os.chdir(script_location)
from ranking_models import rankSVM_column_generation


from sklearn.model_selection import StratifiedKFold
#data=data.values
#class_data=class_data.values

#repetition_states=np.array([5,7,9,11,13])
#no_of_folds=10

           
cross_te_roc_list=[]
cross_tr_roc_list=[]
cross_obj_list=[]

data_location_for_cv=data_location+"/"+d_name+"/test_index/rep"

cntr=0
for i in range(5):
    #skf = StratifiedKFold(n_splits=no_of_folds,random_state=rep_s,shuffle=True)
    #skf.get_n_splits(data, class_data) 
    tmp_data_location=data_location_for_cv+str(i+1)
    os.chdir(tmp_data_location)
    j=0
    for j in range(10):
        
        
        test_index_loc=tmp_data_location+"/test_fold"+str(j)+".csv"
        test_index=pd.read_csv(test_index_loc,sep=',',header=None)
        test_index=test_index.values.reshape(len(test_index))
        test_data, test_class = data.iloc[test_index,:], class_data.iloc[test_index,:]
        
        tr_index_loc=tmp_data_location+"/train_fold"+str(j)+".csv"
        tr_index=pd.read_csv(tr_index_loc,sep=',',header=None)
        tr_index=tr_index.values.reshape(len(tr_index))
        train_data, train_class = data.iloc[tr_index,:], class_data.iloc[tr_index,:]
        
        
        
        df=pd.concat([train_data, train_class], axis=1)
        df_test=pd.concat([test_data,test_class],axis=1)
        
        method1=rankSVM_column_generation(train_data,train_class,test_data,test_class,df,df_test,
                                  distance="euclidian",stopping_condition="obj",
                                  stopping_percentage=0.005,
                                  selected_col_index=0,scale=True,ignore_initial=False,manual_selection=False)
        
        method1.data_preprocess()
        method1.solve_full_model()
        method1.predict_test_full_model()
        #method1.run_model_with_column_generation(plot=False)
        
        
        #cross_te_roc_list.append(method1.test_roc_list)
        #cross_tr_roc_list.append(method1.train_roc_list)
        #cross_obj_list.append(method1.objective_values)
        
        tr_accu_final=method1.ftrain_accuracy
        tr_roc_final=method1.ftrainroc
        te_accu_final=method1.ftest_accu
        te_roc_final=method1.ftestroc        
        if cntr==0:
            cross_val_summary=pd.DataFrame({'Rep_s':i, 'Test_fold_no':j,'Num_features':len(method1.train_data),'stopping_percent':method1.stopping_percentage,'train_accu':tr_accu_final,'train_roc':tr_roc_final,'test_accu':te_accu_final,'test_roc':te_roc_final},index=range(1))
        else:
            tmp=pd.DataFrame({'Rep_s':i, 'Test_fold_no':j,'Num_features':len(method1.train_data),'stopping_percent':method1.stopping_percentage,'train_accu':tr_accu_final,'train_roc':tr_roc_final,'test_accu':te_accu_final,'test_roc':te_roc_final},index=range(1))
            cross_val_summary=cross_val_summary.append(tmp)
        
        del method1
        cntr=cntr+1
        print(cntr)
